In [1]:
%pip install python-dotenv langchain-upstage
%pip install beautifulsoup4
%pip install APScheduler
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from apscheduler.schedulers.blocking import BlockingScheduler
import logging
from datetime import datetime
import requests
import re

# 로깅 설정
logging.basicConfig(filename='crawler.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s')

# 최신 wr_id를 저장할 파일
LAST_ID_FILE = 'last_crawled_id.txt'
docs=None
def get_latest_wr_id():
    url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1"
    response = requests.get(url)
    if response.status_code == 200:
        match = re.search(r'wr_id=(\d+)', response.text)
        if match:
            return int(match.group(1))
    return None

def save_last_crawled_id(wr_id):
    with open(LAST_ID_FILE, 'w') as f:
        f.write(str(wr_id))

def load_last_crawled_id():
    try:
        with open(LAST_ID_FILE, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return None

def crawl_pages():
    global docs
    try:
        latest_wr_id = get_latest_wr_id()
        if latest_wr_id is None:
            logging.error("Fail to bring recent wr_id.")
            return

        last_crawled_id = load_last_crawled_id()
        
        if last_crawled_id is None or latest_wr_id > last_crawled_id:
            start_id = latest_wr_id
            end_id = last_crawled_id if last_crawled_id else latest_wr_id - 10  # 처음 실행 시 최근 10개만 크롤링
            base_url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id="
            
            urls = [f"{base_url}{i}" for i in range(start_id, end_id - 1, -1)]
            
            loader = WebBaseLoader(
                web_paths=urls,
                bs_kwargs=dict(
                    parse_only=bs4.SoupStrainer(
                        "div", attrs={"id": ["bo_v_con", "bo_v_title"]}
                    )
                )
            )
            
            docs = loader.load()
            docs = [doc for doc in docs if len(doc.page_content.strip()) > 0]
            
            if docs:
                logging.info(f"New data by crawlling ({len(docs)} 건):")
                for doc in docs:
                    logging.info(doc.page_content[:100] + "...")  # 로그 길이 제한
                
                save_last_crawled_id(latest_wr_id)
            else:
                logging.info("There's no new data.")
        else:
            logging.info("There's no new notification.")
    
    except Exception as e:
        logging.error(f"Error occurs when crawlling: {str(e)}")

# 스케줄러 인스턴스 생성
scheduler = BlockingScheduler()

# 하루에 한 번 크롤링 실행: 매일 00:00에 크롤링
scheduler.add_job(crawl_pages, 'cron', hour=20, minute=10)

# 프로그램 시작 시 즉시 한 번 실행
scheduler.add_job(crawl_pages, 'date', run_date=datetime.now())

logging.info("Start scheduling.")
scheduler.start()


#### 하루에 한번 크롤링 실행하는 코드

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
#### 5분동안 실행하는데 1분주기로 크롤링함. 이후 멈춘 후 문서 확인하는 코드

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from apscheduler.schedulers.blocking import BlockingScheduler
import logging
from datetime import datetime, timedelta
import requests
import re
import threading

# Logging setup
logging.basicConfig(filename='crawler.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s')

# File to store the latest wr_id
LAST_ID_FILE = 'last_crawled_id.txt'

# Global list to store all crawled documents
all_docs = []

def get_latest_wr_id():
    url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1"
    response = requests.get(url)
    if response.status_code == 200:
        match = re.search(r'wr_id=(\d+)', response.text)
        if match:
            return int(match.group(1))
    return None

def save_last_crawled_id(wr_id):
    with open(LAST_ID_FILE, 'w') as f:
        f.write(str(wr_id))

def load_last_crawled_id():
    try:
        with open(LAST_ID_FILE, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return None

def crawl_pages():
    global all_docs
    try:
        latest_wr_id = get_latest_wr_id()
        if latest_wr_id is None:
            logging.error("Failed to retrieve the latest wr_id.")
            return []

        last_crawled_id = load_last_crawled_id()
        
        if last_crawled_id is None or latest_wr_id > last_crawled_id:
            start_id = latest_wr_id
            end_id = last_crawled_id if last_crawled_id else latest_wr_id - 10  # Crawl latest 10 on first run
            base_url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id="
            
            urls = [f"{base_url}{i}" for i in range(start_id, end_id - 1, -1)]
            
            loader = WebBaseLoader(
                web_paths=urls,
                bs_kwargs=dict(
                    parse_only=bs4.SoupStrainer(
                        "div", attrs={"id": ["bo_v_con", "bo_v_title"]}
                    )
                )
            )
            
            docs = loader.load()
            docs = [doc for doc in docs if len(doc.page_content.strip()) > 0]
            
            if docs:
                logging.info(f"New data crawled ({len(docs)} items):")
                for doc in docs:
                    logging.info(doc.page_content[:100] + "...")  # Limit log length
                
                save_last_crawled_id(latest_wr_id)
                all_docs.extend(docs)  # Add new docs to the global list
                return docs
            else:
                logging.info("No new data found.")
        else:
            logging.info("No new notices available.")
        
        return []
    
    except Exception as e:
        logging.error(f"Error occurred during crawling: {str(e)}")
        return []

def process_crawled_data(docs):
    if docs:
        logging.info(f"Processing {len(docs)} documents...")
        # Add your Pinecone storage logic here if needed
        logging.info("Data processing completed.")
    else:
        logging.info("No new data to process.")

# Create scheduler instance
scheduler = BlockingScheduler()

# Define the job to run every 1 minute
def scheduled_job():
    docs = crawl_pages()
    process_crawled_data(docs)

# Function to stop the scheduler after 5 minutes
def stop_scheduler():
    scheduler.shutdown()
    logging.info("Scheduler stopped after 5 minutes.")
    print("Collected data:")
    for i, doc in enumerate(all_docs, 1):
        print(f"Document {i}:")
        print(doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content)
        print("-" * 50)

# Run crawling every 1 minute
scheduler.add_job(scheduled_job, 'interval', minutes=1)

# Run once immediately when the program starts
scheduler.add_job(scheduled_job, 'date', run_date=datetime.now())

# Schedule the stop_scheduler function to run after 5 minutes
stop_time = datetime.now() + timedelta(minutes=5)
threading.Timer((stop_time - datetime.now()).total_seconds(), stop_scheduler).start()

logging.info("Scheduler started. Crawling will be performed every 1 minute for 5 minutes.")
scheduler.start()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Collected data:
Document 1:

2024학년도 2학기『다문화가정 장학금』장학생 선발 계획을 아래와 같이 안내드리니 대상 학생들은 기한 내 신청하시기 바랍니다.가. 선발대상:「다문화가족지원법」상 다문화 가정 구성원으로 직전학기 성적 1.7이상(4.3기준), 한국장학재단 학자금 지원 구간 8구간 이내 학부 재학생 나. 선발인원: 총 12명 다. 장학금액: 학업장려비 100만원(등록금을 초과...
--------------------------------------------------
Document 2:

2024년 제22회 SW역량검증(TOPCIOT) 정기평가 단체접수 신청자를 대상으로 한 신청자 정보등록을 실시합니다. ※ 참조: 2024년 제22회 SW역량검증(TOPCIT) 정기평가 시행 안내(https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=28212)2024년 제 22회 SW역량검증(TOPCIT)...
--------------------------------------------------
Document 3:

[2024년 컴퓨터학부 재학생 만족도 설문조사 실시] 안녕하십니까, 현재 컴퓨터학부 재학생을 대상으로 실시 중인 '만족도 설문조사' 응답률이 저조하여 아래와 같이 설문조사 기간을 9/30(월) 16시까지 연장하오니, 아직 설문조사를 실시하지 않은 컴퓨터학부 재학생 분들의 많은 참여를 부탁드립니다. 감사합니다. - 설문조사 기간 : 2024. 9. 3.(화...
--------------------------------------------------
Document 4:

1차~4차신청(2024. 9. 11.(수) 오전 까지) 신청한 학생에 대하여 선발이 완료되었으며 개별적으로 선발연락을 완료하였습니다. 이에 아직 미선발된 과목에 한해 5차 모집을 진행합니다. (첨부파일 참조)관심있는 학생들의 많은 지원 바랍니다.  모집일: 2024. 9. 19.(월) 10시 까지 모집방

In [1]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from apscheduler.schedulers.blocking import BlockingScheduler
import logging
from datetime import datetime, timedelta
import requests
import re
import threading

# Logging setup
logging.basicConfig(filename='crawler.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s')

# File to store the latest wr_id and end_id
LAST_ID_FILE = 'last_crawled_id.txt'
END_ID_FILE = 'end_crawled_id.txt'  # End ID를 저장할 파일

# Global list to store all crawled documents
all_docs = []

def get_latest_wr_id():
    url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1"
    response = requests.get(url)
    if response.status_code == 200:
        match = re.search(r'wr_id=(\d+)', response.text)
        if match:
            return int(match.group(1))
    return None

def save_last_crawled_id(wr_id):
    with open(LAST_ID_FILE, 'w') as f:
        f.write(str(wr_id))

def load_last_crawled_id():
    try:
        with open(LAST_ID_FILE, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return None

def save_end_crawled_id(end_id):
    with open(END_ID_FILE, 'w') as f:
        f.write(str(end_id))

def load_end_crawled_id():
    try:
        with open(END_ID_FILE, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return None

def set_initial_end_id():

    initial_end_id = 27726  # 2024.01.01 이후의 첫 공지사항 ID
    save_end_crawled_id(initial_end_id)
    return initial_end_id

def crawl_pages():
    global all_docs
    try:
        latest_wr_id = get_latest_wr_id()
        if latest_wr_id is None:
            logging.error("Failed to retrieve the latest wr_id.")
            return []

        last_crawled_id = load_last_crawled_id()
        end_id = load_end_crawled_id()

        # end_id가 없는 경우, 최초 실행 시 2023년 1월 1일 이후 첫 공지의 wr_id를 end_id로 설정
        if end_id is None:
            end_id = set_initial_end_id()

        if last_crawled_id is None or latest_wr_id > last_crawled_id:
            start_id = latest_wr_id
            # end_id는 고정되어 있으며, start_id는 새로운 공지가 올라올 때마다 최신으로 변경
            base_url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id="
            
            urls = [f"{base_url}{i}" for i in range(start_id, end_id - 1, -1)]
            
            loader = WebBaseLoader(
                web_paths=urls,
                bs_kwargs=dict(
                    parse_only=bs4.SoupStrainer(
                        "div", attrs={"id": ["bo_v_con", "bo_v_title"]}
                    )
                )
            )
            
            docs = loader.load()
            docs = [doc for doc in docs if len(doc.page_content.strip()) > 0]
            
            if docs:
                logging.info(f"New data crawled ({len(docs)} items):")
                for doc in docs:
                    logging.info(doc.page_content[:100] + "...")  # Limit log length
                
                save_last_crawled_id(latest_wr_id)
                all_docs.extend(docs)  # Add new docs to the global list
                return docs
            else:
                logging.info("No new data found.")
        else:
            logging.info("No new notices available.")
        
        return []
    
    except Exception as e:
        logging.error(f"Error occurred during crawling: {str(e)}")
        return []

def process_crawled_data(docs):
    if docs:
        logging.info(f"Processing {len(docs)} documents...")
        # Add your Pinecone storage logic here if needed
        logging.info("Data processing completed.")
    else:
        logging.info("No new data to process.")

# Create scheduler instance
scheduler = BlockingScheduler()

# Define the job to run every 1 minute
def scheduled_job():
    docs = crawl_pages()
    process_crawled_data(docs)

# Function to stop the scheduler after 5 minutes
def stop_scheduler():
    scheduler.shutdown()
    logging.info("Scheduler stopped after 5 minutes.")
    print("Collected data:")
    for i, doc in enumerate(all_docs, 1):
        print(f"Document {i}:")
        print(doc.page_content[:200] + "..." if len(doc.page_content) > 200 else doc.page_content)
        print("-" * 50)

# Run crawling every 1 minute
scheduler.add_job(scheduled_job, 'interval', minutes=1)

# Run once immediately when the program starts
scheduler.add_job(scheduled_job, 'date', run_date=datetime.now())

# Schedule the stop_scheduler function to run after 5 minutes
stop_time = datetime.now() + timedelta(minutes=5)
threading.Timer((stop_time - datetime.now()).total_seconds(), stop_scheduler).start()

logging.info("Scheduler started. Crawling will be performed every 1 minute for 5 minutes.")
scheduler.start()


USER_AGENT environment variable not set, consider setting it to identify your requests.


Collected data:
Document 1:

2024학년도 2학기『다문화가정 장학금』장학생 선발 계획을 아래와 같이 안내드리니 대상 학생들은 기한 내 신청하시기 바랍니다.가. 선발대상:「다문화가족지원법」상 다문화 가정 구성원으로 직전학기 성적 1.7이상(4.3기준), 한국장학재단 학자금 지원 구간 8구간 이내 학부 재학생 나. 선발인원: 총 12명 다. 장학금액: 학업장려비 100만원(등록금을 초과...
--------------------------------------------------
Document 2:

2024년 제22회 SW역량검증(TOPCIOT) 정기평가 단체접수 신청자를 대상으로 한 신청자 정보등록을 실시합니다. ※ 참조: 2024년 제22회 SW역량검증(TOPCIT) 정기평가 시행 안내(https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=28212)2024년 제 22회 SW역량검증(TOPCIT)...
--------------------------------------------------
Document 3:

[2024년 컴퓨터학부 재학생 만족도 설문조사 실시] 안녕하십니까, 현재 컴퓨터학부 재학생을 대상으로 실시 중인 '만족도 설문조사' 응답률이 저조하여 아래와 같이 설문조사 기간을 9/30(월) 16시까지 연장하오니, 아직 설문조사를 실시하지 않은 컴퓨터학부 재학생 분들의 많은 참여를 부탁드립니다. 감사합니다. - 설문조사 기간 : 2024. 9. 3.(화...
--------------------------------------------------
Document 4:

1차~4차신청(2024. 9. 11.(수) 오전 까지) 신청한 학생에 대하여 선발이 완료되었으며 개별적으로 선발연락을 완료하였습니다. 이에 아직 미선발된 과목에 한해 5차 모집을 진행합니다. (첨부파일 참조)관심있는 학생들의 많은 지원 바랍니다.  모집일: 2024. 9. 19.(월) 10시 까지 모집방

In [ ]:
#### 실시간 크롤링 아닌 버전은 아래에

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from apscheduler.schedulers.blocking import BlockingScheduler

def crawl_pages():
    # URL 번호 범위 설정
    start_id = 224
    end_id = 200
    base_url = "https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=28"

    # URL 리스트 생성
    urls = [f"{base_url}{i}" for i in range(start_id, end_id - 1, -1)]

    # WebBaseLoader로 페이지 로드
    loader = WebBaseLoader(
        web_paths=urls,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                "div", attrs={"id": ["bo_v_con", "bo_v_title"]}  # 올바른 id 필터링
            )
        )
    )
    
    docs = loader.load()
    docs = [doc for doc in docs if len(doc.page_content.strip()) > 0]
    
    if docs:
        print("크롤링된 데이터:")
        for doc in docs:
            print(doc.page_content)
    else:
        print("새로운 데이터가 없습니다.")

# 스케줄러 인스턴스 생성
scheduler = BlockingScheduler()

# 하루에 한 번 크롤링 실행: 매일 00:00에 크롤링
scheduler.add_job(crawl_pages, 'cron', hour=0, minute=0)


# 스케줄러 시작
scheduler.start()


# docs의 내용 출력
#for doc in docs:
#    print(f"Content: {doc.page_content}")
#   print(f"Metadata: {doc.metadata}")


2182

In [2]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [3]:
import os
import hashlib
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Pinecone API Key와 인덱스 이름 설정
index_name = 'upstage-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# 문서가 비어있는지 확인
if not all_docs:
    raise ValueError("No documents were loaded.")

for doc in all_docs:
    if not doc.page_content:
        raise ValueError("One or more documents have empty content.")

# 문서 내용의 해시값 생성 함수
def get_doc_hash(doc):
    return hashlib.sha256(doc.page_content.encode('utf-8')).hexdigest()

# Pinecone에서 이미 저장된 문서의 해시값 가져오기
existing_docs_hashes = []  # 여기에 Pinecone에서 가져온 기존 데이터의 해시값을 저장해야 함

# 새로운 문서와 기존 문서 비교
new_docs = []
for doc in all_docs:
    doc_hash = get_doc_hash(doc)
    if doc_hash not in existing_docs_hashes:
        new_docs.append(doc)

# 중복되지 않은 새로운 문서만 저장
if new_docs:
    try:
        if embedding is None:
            raise ValueError("Embedding function is not set")
        
        # 새로운 문서들을 Pinecone에 저장
        database = PineconeVectorStore.from_documents(new_docs, embedding, index_name=index_name)
    except Exception as e:
        print("An error occurred while saving new documents:", str(e))
else:
    print("No new documents to save.")


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [34]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# 데이터를 처음 저장할 때 
index_name='upstage-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

if not all_docs:
    raise ValueError("No documents were loaded.")

for doc in all_docs:
    if not doc.page_content:
        raise ValueError("One or more documents have empty content.")

try:
    if not all_docs or not any(doc.page_content for doc in all_docs):
        raise ValueError("Docs are empty or contain no content")

    if embedding is None:
        raise ValueError("Embedding function is not set")

    # PineconeVectorStore 인스턴스 생성
    database = PineconeVectorStore.from_documents(all_docs, embedding, index_name=index_name)
except Exception as e:
    print("An error occurred:", str(e))

# 이미 저장된 데이터를 사용할 때 
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [47]:
#### 파인콘에 데이터 저장 후 retriever을 통해서 질문 응답하장장

In [1]:
import os
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from pinecone import Pinecone
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_upstage import ChatUpstage


#retriever = database.as_retriever(search_kwargs={'k': 4})
#retriever.invoke(query)


llm=ChatUpstage()
prompt=hub.pull("rlm/rag-prompt")

query='2024학년도 인도 인턴십에 대해서 모든 정보를 알려줘 슈발롬아 말좀 알아쳐먹어라'
qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)
ai_mes=qa_chain.invoke({"query":query})
ai_mes

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langsmith\client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langsmith\client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


NameError: name 'database' is not defined